In [1]:
import pandas as pd
import os
import io

# Pull Emission Data for CBD

In [50]:
data_dir = '/mnt/p/323/3236145/TTR/Model/Outputs/__Shortlist Options/i11.6_pwc/004_Emission/004_Emissions_CBD/Outputs'
results = pd.DataFrame()

for filename in os.listdir(data_dir):
    path = os.path.join(data_dir, filename)
    # name_rows = [5]
    # name = pd.read_csv(path, skiprows = lambda x: x not in name_rows)
    # test = pd.read_csv(path, skiprows=11)
    
    f = io.open(path, mode="r", encoding="utf-8")
    content = f.readlines()
    name = content[4].split(',')[1].strip()
    result = pd.read_csv(path, skiprows=11)
    result['scenario'] = name
    
    results = pd.concat([results, result])
    
results['short_name'] = results['scenario'].apply(lambda x: f"{x[:4]}_{x[8:-5].replace('WHC_','')}")

In [52]:
results.drop('Unnamed: 4',axis=1).set_index('scenario').to_csv("/mnt/p/323/3236145/TTR/Model/Outputs/__Shortlist Options/i11.6_pwc/004_Emission/004_Emissions_CBD/emission_summary.csv")

# Pull PT Travel Time Data Out

In [33]:
%%time

data_dir = '/mnt/p/323/3236145/TTR/Model/Outputs/__Shortlist Options'
t_mtx = pd.DataFrame()

for filename in os.listdir(data_dir):
    path = os.path.join(data_dir, filename)
    if '51' in path and '_Travel_time matrices.xlsx' in path:
        excel = pd.ExcelFile(path)
        sheets = excel.sheet_names

        for sheet in sheets:
            mtx = pd.read_excel(excel, sheet, index_col=0, header=0)
            mtx = mtx.stack().reset_index()
            mtx.columns = ['orig', 'dest', sheet]
            mtx = mtx.set_index(['orig', 'dest'])

            t_mtx = pd.concat([t_mtx, mtx], axis=1)

CPU times: user 1min 53s, sys: 411 ms, total: 1min 53s
Wall time: 2min 8s


In [34]:
t_time_ods = pd.read_csv(f'{data_dir}/python/input/travel-time-zone-pairs.csv')
od_grp_dict = dict(t_time_ods.set_index(['From Zone', 'To Zone'])['Travel Time Group'])
od_dir_dict = dict(t_time_ods.set_index(['From Zone', 'To Zone']).apply(lambda x: (x.From, x.To), axis=1))
od_t_mtx = t_mtx.loc[od_grp_dict.keys()]
od_t_mtx = od_t_mtx.stack().reset_index()
od_t_mtx.columns = ['orig', 'dest', 'peak_scen', 'value']

od_t_mtx['mode_scenario'] = od_t_mtx['peak_scen'].apply(lambda x: x[2:-2])
od_t_mtx['peak'] = od_t_mtx['peak_scen'].apply(lambda x: x[:2])
od_t_mtx['group'] = od_t_mtx.apply(lambda x: od_grp_dict[(x.orig, x.dest)], axis=1)
od_t_mtx['direction'] = od_t_mtx.apply(lambda x: od_dir_dict[(x.orig, x.dest)], axis=1)

od_t_mtx = od_t_mtx.pivot(index=['orig','dest','group', 'direction','peak'], columns='mode_scenario', values='value')
od_t_mtx.to_csv(f"{data_dir}/python/output/travel_time_od_summary.csv")

In [35]:
od_t_mtx.reset_index().groupby('group').agg({'PT_mf33':'min'	PT_mf35	PT_mf37	PT_mf39	PT_mf41	'})

mode_scenario                                                               PT_mf33  \
orig dest group                             direction                peak             
73   91   Between Albany West and Glenfield (Albany West, Glenfield) AM    45.63702   
                                                                     IP    38.43072   
                                                                     PM    45.20456   
     106  Between Albany West and Highbury  (Albany West, Highbury)  AM    55.68201   
                                                                     IP    47.97903   
...                                                                             ...   
243  122  Between CBD and Smales Farm       (CBD, Smales Farm)       IP    19.35038   
                                                                     PM    22.23794   
     125  Between CBD and Takapuna          (CBD, Takapuna)          AM    31.56131   
                                                                     IP    26.02250   
                                                                     PM    29.02327   

mode_scenario                                                               PT_mf35  \
orig dest group                             direction                peak             
73   91   Between Albany West and Glenfield (Albany West, Glenfield) AM    24.52352   
                                                                     IP    24.96352   
                                                                     PM    24.52352   
     106  Between Albany West and Highbury  (Albany West, Highbury)  AM    36.84645   
                                                                     IP    36.52520   
...                                                                             ...   
243  122  Between CBD and Smales Farm       (CBD, Smales Farm)       IP    20.91456   
                                                                     PM    20.47456   
     125  Between CBD and Takapuna          (CBD, Takapuna)          AM    18.58304   
                                                                     IP    19.02304   
                                                                     PM    18.58304   

mode_scenario                                                               PT_mf37  \
orig dest group                             direction                peak             
73   91   Between Albany West and Glenfield (Albany West, Glenfield) AM    45.79318   
                                                                     IP    38.61655   
                                                                     PM    45.85462   
     106  Between Albany West and Highbury  (Albany West, Highbury)  AM    55.78659   
                                                                     IP    48.13311   
...                                                                             ...   
243  122  Between CBD and Smales Farm       (CBD, Smales Farm)       IP    18.04926   
                                                                     PM    17.75592   
     125  Between CBD and Takapuna          (CBD, Takapuna)          AM    19.82626   
                                                                     IP    25.02539   
                                                                     PM    19.82626   

mode_scenario                                                               PT_mf39  \
orig dest group                             direction                peak             
73   91   Between Albany West and Glenfield (Albany West, Glenfield) AM    46.16450   
                                                                     IP    38.91262   
                                                                     PM    45.76068   
     106  Between Albany West and Highbury  (Albany West, Highbury)  AM    56.23201   
                                                                     IP    48.4257

# Combine Trip Matrices into One

In [127]:
%%time

data_dir = '/mnt/p/323/3236145/TTR/Model/Outputs/__Shortlist Options/i11.6_pwc/Band_of_Matrices'
results = pd.DataFrame()
# return_mtx = ['CarP']
return_mtx = ['PT Matrix']

for path, subdirs, files in os.walk(data_dir):
    for name in files:
        mtx_path = os.path.join(path, name)
        if any([mtx in mtx_path for mtx in return_mtx]):
            f = io.open(mtx_path, mode="r", encoding="utf-8")
            content = f.readlines()
            
            scen_name = content[5].split(',')[1].strip()
            mtx_name = content[8].split('/')[-1].replace('.csv','').strip()
            
            result = pd.read_csv(mtx_path, skiprows=10, index_col=0)
            result = result.stack().reset_index()
            result.columns = ['orig', 'dest', mtx_name]
            result = result.loc[result[mtx_name] != 0]
            result = result.set_index(['orig', 'dest'])

            results = pd.concat([results, result], axis=1)

CPU times: user 14.1 s, sys: 268 ms, total: 14.4 s
Wall time: 41.1 s


In [129]:
%%time

# results.to_csv(f"{data_dir}/car_person_matrices_ijv.csv")
results.fillna(0).to_csv(f"{data_dir}/PT_person_matrices_ijv.csv")

CPU times: user 8.4 s, sys: 333 ms, total: 8.73 s
Wall time: 33.8 s


# Analyse CBD Trips

In [2]:
%%time

data_dir = '/mnt/p/323/3236145/TTR/Model/Outputs/__Shortlist Options/i11.6_pwc/Band_of_Matrices'
car_mtx = pd.read_csv(f'{data_dir}/car_person_matrices_ijv.csv')
pt_mtx = pd.read_csv(f'{data_dir}/PT_person_matrices_ijv.csv')

CPU times: user 7.78 s, sys: 717 ms, total: 8.49 s
Wall time: 33.6 s


In [40]:
def summary_by_dest_area(from_zones, to_zones, peak, mtx, prefix):
    
    peak_cols = [col for col in mtx.columns if f'{peak} ' in col]
    
    sum_cols = dict()
    for col in peak_cols:
        sum_cols[f'{prefix}{col[:4]}'] = sum_cols.get(f'{prefix}{col[:4]}', []) + [col]

    peak_to_area_mtx = (
        mtx
        .loc[(mtx['dest'].isin(to_zones)) & (mtx['orig'].isin(from_zones))]
        [['orig', 'dest']+peak_cols]
        .reset_index(drop=True)
    )

    for s_col, col in sum_cols.items():
        peak_to_area_mtx[s_col] = peak_to_area_mtx[col].sum(axis=1)

    peak_to_area = peak_to_area_mtx.sum()
    return peak_to_area_mtx, peak_to_area[sum_cols.keys()]

In [41]:
north_zones = [50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134]
cbd_zones = [243, 244, 245, 246, 247, 248, 249, 250, 251]
data_dir = '/mnt/p/323/3236145/TTR/Model/Outputs/__Shortlist Options/i11.6_pwc/Band_of_Matrices'

am_to_cbd_car_mtx, am_to_cbd_car = summary_by_dest_area(north_zones, cbd_zones, 'AM', car_mtx, 'car_person_')
am_to_cbd_pt_mtx, am_to_cbd_pt = summary_by_dest_area(north_zones, cbd_zones, 'AM', pt_mtx, 'pt_person_')
am_to_cbd = pd.concat([am_to_cbd_car, am_to_cbd_pt])

am_to_cbd.to_csv(f"{data_dir}/car_pt_person_north_to_cbd.csv")

In [27]:
mtx['mf3271 AM PT Matrix'].sum()

174522.36754611347

In [28]:
mtx[['mf3271 AM PT Matrix']].sum(axis=1).sum()

174522.36754611347